In [1]:
import Pkg; 
Pkg.activate(@__DIR__); 
Pkg.instantiate()

  Activating project at `~/art/spline-trajectory-optimization/julia`


In [2]:
include("utils/SplineTrajectory.jl")
include("utils/DiscreteTrajectory.jl")
include("utils/IO.jl")

load_raw_trajectory (generic function with 1 method)

In [3]:
import ForwardDiff as FD
import BSplineKit as BK
import PyPlot as plt
plt.pygui(true)
using Test
using DelimitedFiles
using OSQP
using SparseArrays
using Statistics
Drawing()

In [6]:
interval = 3.0 # m

# x, y, z = load_raw_trajectory("examples/race_track/putnam/PUTNAM_A_CENTER_enu.csv")
# traj_s = SplineTrajectory(x, y, 3)

# x, y, z = load_raw_trajectory("examples/race_track/putnam/PUTNAM_BOUND_A_LEFT_enu.csv")
# left_s = SplineTrajectory(x, y, 3)

# x, y, z = load_raw_trajectory("examples/race_track/putnam/PUTNAM_BOUND_A_RIGHT_enu.csv")
# right_s = SplineTrajectory(x, y, 3)

x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_UNOPTIMIZED_LINE_enu.csv")
traj_s = SplineTrajectory(x, y, 3)

x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_LEFT_BOUNDARY_enu.csv")
left_s = SplineTrajectory(x, y, 3)

x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_RIGHT_BOUNDARY_enu.csv")
right_s = SplineTrajectory(x, y, 3)

traj_d, ts= discretize_trajectory(traj_s, interval, false)
left_d, _ = discretize_trajectory(left_s, 0.5)
right_d, _ = discretize_trajectory(right_s, 0.5)

set_trajectory_bounds(traj_d, left_d, right_d)

# calculate distance to left and right boundary
traj_tum = zeros(size(traj_d.data, 1), 4)
traj_tum[:, 1:2] = traj_d.data[:, 1:2]
left_diff = traj_d.data[:, TRAJ_LEFT_BOUND_X:TRAJ_LEFT_BOUND_Y] .- traj_d.data[:, TRAJ_X:TRAJ_Y]
right_diff = traj_d.data[:, TRAJ_RIGHT_BOUND_X:TRAJ_RIGHT_BOUND_Y] .- traj_d.data[:, TRAJ_X:TRAJ_Y]
traj_tum[:, 3] = vec(sqrt.(right_diff[:, 1].^2 .+ right_diff[:, 2].^2))
traj_tum[:, 4] = vec(sqrt.(left_diff[:, 1].^2 .+ left_diff[:, 2].^2))

# save trajectory to csv
writedlm("experiment_data/traj_tum.csv", traj_tum, ',')
